In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from tesserocr import PyTessBaseAPI
import os
from PIL import Image
import json

In [2]:
data = []

In [3]:
img =  cv2.imread(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\Data-storage\form-5.jpg")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

thresh_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

# Blur the image
blur = cv2.GaussianBlur(thresh_inv,(1,1),0)

thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

# find contours
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]



In [4]:
for cnt in cnts: 
    approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    
    if len(approx) == 4: 
        coord_and_data = []
        
        x, y, w, h = cv2.boundingRect(cnt)

        c1 = y
        c2 = y+h
        c3 = x
        c4 = x+w

        # Cropping
        image_arr = img
        image_arr = image_arr[c1:c2, c3:c4]
        
        #plt.imshow(image_arr)
        #plt.show()
        
        # Saving coordinates
        coord_and_data.append([c1, c2, c3, c4])

        # Temporary saving image for OCR processing
        im = Image.fromarray(image_arr)
        im.save(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\temp\image_arr.jpeg")

        # OCR process
        with PyTessBaseAPI(path=r'C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\Tessdata', lang='eng_best') as api:
            api.SetImageFile(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\temp\image_arr.jpeg")
            coord_and_data.append([api.GetUTF8Text()])
        
        # Deleting image    
        os.remove(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\temp\image_arr.jpeg") 
        
        # Saving data of text
        data.append(coord_and_data)

In [5]:
print(data)
print(thresh.shape)

[[[230, 255, 281, 356], [' \n']], [[102, 116, 474, 477], ['']], [[35, 67, 264, 630], ['']]]
(540, 960)


In [6]:
template = {
  "image_dimension": thresh.shape,
  "data_fields": data,
}

template_json = json.dumps(template)

with open("template.json", "w") as outfile:
    json.dump(template_json, outfile)